In [1]:
import functools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from torchvision import datasets, transforms
from collections import deque
#from ULogicalModels import UDecisionTreeClassifier
#from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, ConfusionMatrixDisplay, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tqdm as tqdm

In [37]:
def getModelError(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    y_predict = model.predict(X_test)
    return classification_report(y_test, y_predict)
    
def compareModels(model, uModel, X, y, is_need_scale):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)
    if is_need_scale:
        scaler = StandardScaler()
        scaler.fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)
    model_error = getModelError(model, X_train, X_test, y_train, y_test)
    uModel_error = getModelError(uModel, X_train, X_test, y_train, y_test)
    print('Ошибка на пакетной модели\n', model_error)
    print('Ошибка на реализованной модели\n', uModel_error)

In [38]:
#Вины
data_1 = pd.read_csv('DATA/wine_fraud.csv')
X = data_1.drop(['type','quality'], axis=1)
y = data_1['type']
y = y.map({'red': 1, 'white': -1})

In [8]:
#Исследование слуха
data_1 = pd.read_csv('DATA/hearing_test.csv')
X = data_1.drop('test_result', axis=1)
y = data_1['test_result']

In [2]:
#Ирисы
data_1 = pd.read_csv('DATA/iris.csv')
X = data_1.drop('species', axis=1)
y = data_1['species']

In [35]:
model = AdaBoostClassifier(n_estimators=10)
uModel = UAdaBoost()

In [39]:
compareModels(model, uModel, X, y, False)

Ошибка на пакетной модели
               precision    recall  f1-score   support

          -1       0.99      0.99      0.99      1451
           1       0.98      0.97      0.97       499

    accuracy                           0.99      1950
   macro avg       0.98      0.98      0.98      1950
weighted avg       0.99      0.99      0.99      1950

Ошибка на реализованной модели
               precision    recall  f1-score   support

          -1       0.99      0.99      0.99      1451
           1       0.97      0.98      0.97       499

    accuracy                           0.99      1950
   macro avg       0.98      0.98      0.98      1950
weighted avg       0.99      0.99      0.99      1950



C:\Users\SMD\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
